In [136]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")

In [137]:
#get all stock data
FILEIN = 'midterm_1_data.xlsx'
sheet_exrets = 'excess returns'
sheet_spy = 'spy'

retsx = pd.read_excel(FILEIN, sheet_name=sheet_exrets).set_index('date')
spy = pd.read_excel(FILEIN, sheet_name=sheet_spy).set_index('date')

# retsx.drop('AAPL', axis=1, inplace=True)

In [138]:
spy.head()

,SPY
date,
2016-01-15,-0.021430
2016-01-22,0.014429
2016-01-29,0.016801
2016-02-05,-0.029789
2016-02-12,-0.007023


In [139]:
retsx.head()

,AAPL,MSFT,AMZN,NVDA,GOOGL,TSLA,XOM
date,,,,,,,
2016-01-15,-0.006077,-0.033445,-0.068591,-0.092883,-0.035777,-0.036346,0.030855
2016-01-22,0.045278,0.026605,0.047061,0.050539,0.050316,-0.010817,-0.011908
2016-01-29,-0.051720,0.042053,-0.027223,0.018024,0.009834,-0.067483,0.005221
2016-02-05,-0.036236,-0.096825,-0.151901,-0.104974,-0.082989,-0.156939,0.021309
2016-02-12,-0.007887,-0.000785,0.002275,-0.034052,-0.003101,-0.078688,0.013486


ALLOCATION!

In [140]:
#this function calculates the statistics we commonly refer to when analyzing the portfolio
def performance_summary(return_data, annualization = 12):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*annualization)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(annualization))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']
    
    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    summary_stats['Min'] = return_data.min()
    summary_stats['Max'] = return_data.max()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats


summary_stats_retsx = performance_summary(retsx, 52)
summary_stats_retsx

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05),Min,Max,Max Drawdown,Peak,Bottom,Recovery
AAPL,0.319421,0.283883,1.125183,-0.334342,2.672198,-0.052313,-0.085612,-0.190566,0.143562,-0.372094,2018-10-05,2019-01-04,2019-11-08
MSFT,0.288087,0.240206,1.199334,-0.359175,1.737189,-0.049366,-0.071559,-0.150492,0.104231,-0.299537,2020-02-14,2020-03-20,2020-07-03
AMZN,0.239457,0.310389,0.771474,-0.210630,1.746315,-0.061868,-0.096065,-0.151901,0.156111,-0.468127,2021-07-09,2023-01-06,NaT
NVDA,0.650658,0.468096,1.390011,0.425676,2.244417,-0.083805,-0.119446,-0.210199,0.332580,-0.592344,2021-11-19,2022-10-14,2023-05-19
GOOGL,0.193328,0.274217,0.705020,0.041986,1.143573,-0.055729,-0.078408,-0.135524,0.149258,-0.348297,2022-03-25,2023-01-06,NaT
TSLA,0.569728,0.607026,0.938556,0.441455,1.527376,-0.122519,-0.155313,-0.284957,0.334897,-0.682185,2021-11-05,2023-01-06,NaT
XOM,0.124196,0.311613,0.398557,0.097936,3.129459,-0.061685,-0.097340,-0.175338,0.184173,-0.671435,2016-12-16,2020-03-20,2022-03-11


In [141]:
max_sharpe_ticker = summary_stats_retsx['Sharpe Ratio'].idxmax()
max_sharpe_value = summary_stats_retsx['Sharpe Ratio'].max()
min_sharpe_ticker = summary_stats_retsx['Sharpe Ratio'].idxmin()
min_sharpe_value = summary_stats_retsx['Sharpe Ratio'].min()

print(f"The highest Sharpe ratio is {max_sharpe_value:.2f}, which comes from {max_sharpe_ticker}.")
print(f"The lowest Sharpe ratio is {min_sharpe_value:.2f}, which comes from {min_sharpe_ticker}.")

The highest Sharpe ratio is 1.39, which comes from NVDA.
The lowest Sharpe ratio is 0.40, which comes from XOM.


In [142]:
#uses matrix math to calculate covariance, and find the weights that maximize risk/return

def tangency_weights(returns, cov_mat = 1):
    
    if cov_mat ==1:
        cov_inv = np.linalg.inv((returns.cov()*12))
    else:
        cov = returns.cov()
        covmat_diag = np.diag(np.diag((cov)))
        covmat = cov_mat * cov + (1-cov_mat) * covmat_diag
        cov_inv = np.linalg.inv((covmat*12))  
        
    # ones = np.ones(returns.columns[1:].shape) 
    ones = np.ones((returns.shape[1], 1))

    # mu = returns.mean()*12
    mu = returns.mean().values.reshape(-1, 1)  # Convert mu to a column vector

    # print("ones shape:", ones.shape)
    # print("cov_inv shape:", cov_inv.shape)
    # print("mu shape:", mu.shape)
    
     # scaling = 1/(np.transpose(ones) @ cov_inv @ mu)
    scaling = 1 / (ones.T @ cov_inv @ mu)  # Transpose ones to make it a row vector
    
    tangent_return = scaling*(cov_inv @ mu)

    # tangency_wts = pd.DataFrame(index = returns.columns[1:], data = tangent_return, columns = ['Tangent Weights'] )
    tangency_wts = pd.DataFrame(index = returns.columns[:], data = tangent_return[:], columns = ['Tangent Weights'] )
        
    return tangency_wts

w_t = tangency_weights(retsx.reset_index(drop=True), cov_mat=1)
w_t

,Tangent Weights
AAPL,0.322605
MSFT,0.787496
AMZN,-0.228607
NVDA,0.495996
GOOGL,-0.502721
TSLA,0.105975
XOM,0.019257


In [143]:
#applies the weights to our old returns to get weighted returns and then recalculates the summary statistics

w_tan_summary_statistics = performance_summary(retsx @ w_t , 52)
w_tan_summary_statistics

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05),Min,Max,Max Drawdown,Peak,Bottom,Recovery
Tangent Weights,0.563474,0.358351,1.572409,0.00438,1.899066,-0.063723,-0.096174,-0.223741,0.193774,-0.383571,2018-09-14,2019-01-04,2019-11-08


In [144]:
retsx @ w_t

,Tangent Weights
date,
2016-01-15,-0.043959
2016-01-22,0.023196
2016-01-29,0.019600
2016-02-05,-0.079781
2016-02-12,-0.027092
...,...
2023-06-16,0.092836
2023-06-23,-0.025987
2023-06-30,0.043890


In [145]:
lowestSR = summary_stats_retsx['Sharpe Ratio'].idxmin()
lowestSRwt = w_t.loc[lowestSR][0]

lowestWT = w_t.idxmin()[0]
lowestWTsr = summary_stats_retsx.loc[lowestWT]['Sharpe Ratio']

print(f"The lowest sharpe ratio is {lowestSR} and is weighted {lowestSRwt:.2f} in the tangency portfolio.")
print(f"The lowest weight is {lowestWT} and has sharpe ratio of {lowestWTsr:.2f}.")

#This shows that all we care about is CORRELATION in a portfolio!

The lowest sharpe ratio is XOM and is weighted 0.02 in the tangency portfolio.
The lowest weight is GOOGL and has sharpe ratio of 0.71.


In [146]:
#calculating out of sample performance now
#calculating new weights unscaled

retsx_IS = retsx.loc[:'2022']
retsx_OOS = retsx.loc['2023':]

wts = pd.DataFrame(index = retsx_IS.columns, columns=['tangency','equal weights','regularized'])

wts.loc[:,'tangency'] = tangency_weights(retsx_IS, cov_mat = 1).values
wts.loc[:,'equal weights'] = 1/len(retsx_IS.columns)
wts.loc[:,'regularized'] = tangency_weights(retsx_IS, cov_mat = (1/3)).values #regularizing by a factor of 1/3

wts

,tangency,equal weights,regularized
AAPL,0.310565,0.142857,0.237267
MSFT,1.073114,0.142857,0.330835
AMZN,-0.259080,0.142857,0.047178
NVDA,0.380133,0.142857,0.196774
GOOGL,-0.751548,0.142857,0.011382
TSLA,0.101559,0.142857,0.09017
XOM,0.145257,0.142857,0.086393


In [147]:
#calculating weights scaled (not sure if i need this)

# wts_scaled = wts.copy()
# wts_scaled *= (retsx_IS.mean()@wts_scaled)

# wts_scaled

# performance_summary(retsx_IS @ wts_scaled, 52)
# performance_summary(retsx_OOS @ wts_scaled, 52)

In [148]:
#calculating IS performance summaries
performance_summary(retsx_IS @ wts, 52)


,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05),Min,Max,Max Drawdown,Peak,Bottom,Recovery
tangency,0.471913,0.331179,1.424947,-0.120732,3.033074,-0.055151,-0.092073,-0.231313,0.195057,-0.373375,2020-02-14,2020-03-20,2020-06-05
equal weights,0.293432,0.260804,1.125106,-0.297489,1.712386,-0.052046,-0.075865,-0.153175,0.112469,-0.353207,2020-02-14,2020-03-20,2020-06-05
regularized,0.325593,0.260874,1.248085,-0.366472,1.902271,-0.049572,-0.076368,-0.161853,0.109635,-0.342349,2020-02-14,2020-03-20,2020-06-05


In [149]:
#calculating OS performance summaries

performance_summary(retsx_OOS @ wts, 52)

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05),Min,Max,Max Drawdown,Peak,Bottom,Recovery
tangency,1.204709,0.443716,2.715043,-0.122475,0.000357,-0.080687,-0.102946,-0.110876,0.135754,-0.110876,2023-05-05,2023-05-12,2023-05-26
equal weights,0.955133,0.246953,3.867668,-0.181519,1.026404,-0.03576,-0.056245,-0.069187,0.094587,-0.069187,2023-03-03,2023-03-10,2023-03-31
regularized,1.013509,0.250254,4.049917,-0.160608,0.400467,-0.041306,-0.05573,-0.060332,0.088543,-0.060332,2023-03-03,2023-03-10,2023-03-24


In [156]:
#calculating the Global Minimum Variance portfolio (GMV)

def gmv_weights(tot_returns):
    
    ones = np.ones((tot_returns.shape[1], 1))
    cov = tot_returns.cov()*12
    cov_inv = np.linalg.inv(cov)
    scaling = 1/(np.transpose(ones) @ cov_inv @ ones)
    gmv_tot = scaling * cov_inv @ ones
    gmv_wts = pd.DataFrame(index = tot_returns.columns[:], data = gmv_tot[:], columns = ['GMV Weights'] )

    return gmv_wts

gmv_weights(retsx)

,GMV Weights
AAPL,0.206231
MSFT,0.491250
AMZN,0.160866
NVDA,-0.119168
GOOGL,0.011378
TSLA,-0.046927
XOM,0.296369


In [158]:
w_tan_summary_statistics[['Mean']]
#If the target mean is above the tangency mean, we must short the GMV. If target mean is below tangency mean, then we long the GMV. In this case, we are long the GMV.

,Mean
Tangent Weights,0.563474


PERFOMANCE!

In [162]:
#kurtosis and skew
retsx['TSLA'].agg(['skew','kurtosis'])

#positive = more positive return days, and fatter tails

skew        0.441455
kurtosis    1.527376
Name: TSLA, dtype: float64

In [167]:
#drawdown

wealth = (1+retsx).cumprod()
wealth_max = wealth.cummax()
drawdown = wealth / wealth_max - 1
max_drawdown = drawdown.min()
max_drawdown['TSLA']

-0.6821852296331565

In [173]:
#beta, and many ratios
model = sm.OLS(retsx['TSLA'], sm.add_constant(spy), missing= 'drop').fit()
summary = model.params.to_frame('Summary').T
summary.columns = ['Alpha', 'Beta']

summary['Sortino Ratio'] = retsx['TSLA'].mean() / retsx['TSLA'][retsx['TSLA'] < 0].std() * np.sqrt(52) #excess return over negative std dev
summary['Information Ratio'] = summary['Alpha'] / model.resid.std() * np.sqrt(52) #excess returns (to market) over tracking error
summary['Treynor Ratio'] = retsx['TSLA'].mean() / summary['Beta'] * 52 #excess returns over Beta
summary['Alpha'] = summary['Alpha'] * 52

summary


,Alpha,Beta,Sortino Ratio,Information Ratio,Treynor Ratio
Summary,0.30947,1.776825,1.642329,0.596055,0.320644


In [177]:
#VaR calculations

#cVar at the 5ht percentile
cvar = -stats.norm.pdf(1.65) / .05 * retsx['TSLA'].std()
cvar

-0.17217191106804966

In [183]:
#5th percentile, one period ahead VaR, rolling on m=52

var = -1.65 * retsx['TSLA'].rolling(52).std().shift(1).dropna()
var

date
2017-01-13   -0.099884
2017-01-20   -0.099711
2017-01-27   -0.099878
2017-02-03   -0.098615
2017-02-10   -0.090954
                ...   
2023-06-16   -0.157886
2023-06-23   -0.157712
2023-06-30   -0.155459
2023-07-07   -0.153663
2023-07-14   -0.152046
Name: TSLA, Length: 340, dtype: float64

In [187]:
#with rolling volatility

rol_vol = np.sqrt((retsx['TSLA']**2).rolling(52).mean().shift())
var_rol = -1.65 * rol_vol.dropna()
var_rol

date
2017-01-13   -0.099069
2017-01-20   -0.099049
2017-01-27   -0.099337
2017-02-03   -0.098448
2017-02-10   -0.091687
                ...   
2023-06-16   -0.156624
2023-06-23   -0.156740
2023-06-30   -0.154200
2023-07-07   -0.152694
2023-07-14   -0.150967
Name: TSLA, Length: 340, dtype: float64

In [189]:
#hit ratio

(retsx['TSLA'].loc[var.index] < var).mean()

0.05588235294117647

In [190]:
(retsx['TSLA'].loc[var_rol.index] < var_rol).mean()

0.05588235294117647

HEDGING

In [217]:
#hedge nvidia with other stocks

y = retsx['NVDA']   
X = sm.add_constant(retsx[['AAPL','AMZN','GOOGL','MSFT']])
model = sm.OLS(y,X).fit()

betas = model.params

exposure = betas * 100_000_000
exposure = exposure.drop('const')
exposure = pd.DataFrame(exposure, columns=['NVDA'])
exposure.index = exposure.index.str.replace('Beta', '')
exposure.loc['Total'] = exposure.sum()
exposure['NVDA'] = exposure['NVDA'].map('${:,.0f}'.format)

exposure


,NVDA
AAPL,"$34,168,649"
AMZN,"$41,725,986"
GOOGL,"$-784,795"
MSFT,"$58,789,673"
Total,"$133,899,513"


In [244]:
rsquared = model.rsquared
predicitons = model.predict(X)
residuals = y - predicitons
tracking_error = np.sqrt(np.mean(residuals**2))

metrics = pd.DataFrame({ 'R-squared': [rsquared],'Tracking Error': [tracking_error]}).T
metrics.columns = ['NVDA']

metrics

,NVDA
R-squared,0.458168
Tracking Error,0.047721


In [243]:
intercept = model.params['const'] * 52

alpha_df = pd.DataFrame({'Alpha': [intercept]}, index=['NVDA'])
alpha_df

,Alpha
NVDA,0.273752


Misc Formulas

In [1]:
def time_series_regression(portfolio, factors, FF3F = False, resid = False):
    
    ff_report = pd.DataFrame(index=portfolio.columns)
    bm_residuals = pd.DataFrame(columns=portfolio.columns)

    rhs = sm.add_constant(factors)

    for portf in portfolio.columns:
        lhs = portfolio[portf]
        res = sm.OLS(lhs, rhs, missing='drop').fit()
        ff_report.loc[portf, 'alpha_hat'] = res.params['const'] * 12
        ff_report.loc[portf, 'beta_mkt'] = res.params[1]
        if FF3F:
            ff_report.loc[portf, 'Size beta'] = res.params[2] 
            ff_report.loc[portf, 'Value beta'] = res.params[3]
            
        ff_report.loc[portf, 'info_ratio'] = np.sqrt(12) * res.params['const'] / res.resid.std()
        ff_report.loc[portf, 'treynor_ratio'] = 12 * portfolio[portf].mean() / res.params[1]
        ff_report.loc[portf, 'R-squared'] = res.rsquared
        ff_report.loc[portf, 'Tracking Error'] = (res.resid.std()*np.sqrt(12))

        if resid:
            bm_residuals[portf] = res.resid
            
            
        
    if resid:
        return bm_residuals
        
    return ff_report

In [ ]:
def mv_portfolio(target_ret, tot_returns):
    
    mu_tan = tot_returns.mean() @ tangency_weights(tot_returns, cov_mat = 1)
    mu_gmv = tot_returns.mean() @ gmv_weights(tot_returns)
    
    delta = (target_ret - mu_gmv[0])/(mu_tan[0] - mu_gmv[0])
    mv_weights = (delta * tangency_weights(tot_returns, cov_mat = 1)).values + ((1-delta)*gmv_weights(tot_returns)).values
    
    MV = pd.DataFrame(index = tot_returns.columns[1:], data = mv_weights, columns = ['MV Weights'] )
    MV['tangency weights'] =  tangency_weights(tot_returns, cov_mat = 1).values
    MV['GMV weights'] =   gmv_weights(tot_returns).values


    return MV
